In [499]:
import csv
import pandas as pd
import numpy as np

In [500]:
headers =  ['age', 'sex','chest_pain','resting_blood_pressure',  
        'serum_cholestoral', 'fasting_blood_sugar', 'resting_ecg_results',
        'max_heart_rate_achieved', 'exercise_induced_angina', 'oldpeak',"slope of the peak",
        'num_of_major_vessels','thal', 'heart_disease']
heart_df = pd.read_csv('heart.dat', sep=' ',names=headers)

In [501]:
heart_df.head()

,age,sex,chest_pain,resting_blood_pressure,serum_cholestoral,fasting_blood_sugar,resting_ecg_results,max_heart_rate_achieved,exercise_induced_angina,oldpeak,slope of the peak,num_of_major_vessels,thal,heart_disease
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0,2
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0,1
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,2
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0,1
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,1


In [502]:
heart_data = heart_df.to_numpy()

# heart_data[heart_data == 0] = 0.0000001
# # setting all values equal to 0 to a small value for log and division errors

In [503]:
heart_data.shape

(270, 14)

In [504]:
np.random.shuffle(heart_data)
train_size = int(0.8 * heart_data.shape[0])
heart_train = heart_data[:train_size,:]
heart_test= heart_data[train_size:,:]

In [505]:
train_features= heart_train[:,:heart_train.shape[1]-1]
train_target = heart_train[:,heart_train.shape[1]-1]


In [506]:
test_features= heart_test[:,:heart_test.shape[1]-1]
test_target = heart_test[:,heart_test.shape[1]-1]

In [507]:
for i in range(train_target.shape[0]):
    if train_target[i] == 1:
        train_target[i]=0
    else:
        train_target[i]=1
        
for i in range(test_target.shape[0]):
    if test_target[i] == 1:
        test_target[i]=0
    else:
        test_target[i]=1

In [513]:
nn_input = heart_data.shape[1]-1
classes = np.unique(heart_data[:,heart_data.shape[1]-1])
nn_output = len(classes)-1

In [514]:
# heart_df = pd.read_csv('breastcancer.csv')
# heart_data = heart_df.to_numpy()
# target=heart_data[:,1]
# target_modified = np.ones(target.shape)
# for i in range(target.shape[0]):
#     if target[i] == 'B':
#         target_modified[i] = 0
# target=target_modified
# feature=np.concatenate((heart_data[:,0].reshape(569,1),heart_data[:,2:]),axis=1)

# train_size = int(0.8 * heart_data.shape[0])

# train_features= feature[:train_size,:]
# train_target = target[:train_size]
# test_features= feature[train_size:,:]
# test_target = target[train_size:]
# nn_input = heart_data.shape[1]-1
# nn_output = 1

In [516]:
class ann :
    def __init__(self,nn_inputs,hidden_l,nn_output):
        self.nn_inputs = nn_inputs
        self.hidden_l = hidden_l  #list contains number of nodes in each hidden layer
        self.nn_output = nn_output
        self.bias = []
        self.weights=[] # e.g: W1.shape=(nn_input,nn_hidden[0]);   W2.shape=(nn_hidden[0],nn_hidden[1]) .....
        self.calculated_params =[]
        self.learning_rate = 0.2
        
        
        np.random.seed(1) 
        self.weights.append(np.random.randn(self.nn_inputs,self.hidden_l[0]) )  #weights initilisation
        for i in range(len(self.hidden_l)-1):
            self.weights.append(np.random.randn(self.hidden_l[i], self.hidden_l[i+1]) ) 
        self.weights.append(np.random.randn(self.hidden_l[len(self.hidden_l)-1],self.nn_output) ) 
       
        for i in range(len(self.hidden_l)):          #Bias initialisation 
            self.bias.append(np.random.randn(self.hidden_l[i]))
        self.bias.append(np.random.randn(self.nn_output))
        
        
        
        for i in range(len(self.weights)):
            print(self.weights[i].shape)
        print(self.bias)
        
    def min_val(self, x):
        min_val= 0.0000000001
        return np.maximum(x, min_val)
                  
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def relu(self,x):
        return np.maximum(0,x)
    
    def sigmoid_deriv(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    def tanh_deriv(self,x):
        return 1.-np.tanh(x)**2
    
    
    def ce_loss(self,target,predicted):
        n_samples = len(target)
        loss=0
#         for e in range(n_samples):
#             if predicted[e] == 0.0 :
#                 predictd[e] = 0.000000001   #to avoid passing 0 to log
        loss = -1/n_samples * (np.sum(np.multiply(np.log(self.min_val(predicted)),target )) + np.sum(np.multiply(np.log(self.min_val(1.-predicted)),1.-target )))
                                            #loss case 2 output classes (needs to be generalised)
        return loss
    
    
    def activation(self,index,x):
        return x.dot(self.weights[index]) + self.bias[index]
    
    def forward_prop(self,x,y):
        result=x
        for i in range(len(self.weights)-1):                    # activations of hidden layers 
            dot = self.activation(i,result)
            result= np.tanh(dot)
            self.calculated_params.append(dot)
            self.calculated_params.append(result)
#         self.calculated_params.append(self.activation(len(self.weights)-1,result))
        result=self.sigmoid(self.activation(len(self.weights)-1,result))  #activation of output layer
        
        loss = self.ce_loss(y,result)
        
        return result,loss
        
        
    def back_prop(self,x,l,y,y_pred):
        z_deriv = []
        a_deriv = []
        w_deriv = []
        b_deriv = []
#         for e in y_pred:
#             if e ==0:
#                 e = 0.0000001
            
        dl_wrt_y_pred = np.divide(1-y,self.min_val(1-y_pred)) - np.divide(y, self.min_val(y_pred))  # case of losss of 2 classes  
        dl_wrt_sig = y_pred * (1-y_pred)
        dl_wrt_y_pred_z = dl_wrt_y_pred.dot(dl_wrt_sig)
        z_deriv.append(dl_wrt_y_pred_z)
        
        for i in range(0,len(self.weights)-1):
            dl_wrt_A =  z_deriv[-1].dot((self.weights[len(self.weights)-1-i]).T)
            a_deriv.append(dl_wrt_A)

            dl_wrt_w = self.calculated_params[len(self.calculated_params)-1-2*i].T.dot(z_deriv[-1])
            w_deriv.append(dl_wrt_w)

            dl_wrt_b = np.sum(z_deriv[-1], axis=0, keepdims=True)
            b_deriv.append(dl_wrt_b)

            dl_wrt_z = a_deriv[-1] * self.tanh_deriv(self.calculated_params[len(self.calculated_params)-1-(2*i+1)])
            z_deriv.append(dl_wrt_z)
            
        dl_wrt_b = np.sum(z_deriv[-1], axis=0, keepdims=True)
        b_deriv.append(dl_wrt_b)
        
        dl_wrt_w = x.T.dot(z_deriv[-1])
        w_deriv.append(dl_wrt_w)
        
            
        #updating weights and bias
        
        for i in range(len(self.weights)):
            self.weights[i] = self.weights[i] - self.learning_rate * w_deriv[len(self.weights)-1-i]
            self.bias[i] = self.bias[i] - self.learning_rate * b_deriv[len(self.weights)-1-i]

        
    def train(self,x,y):
        last_loss=0
        loss=1
        while(loss != last_loss):
            last_loss=loss
            predicted,loss = self.forward_prop(x,y)
            self.back_prop(x,loss,y,predicted)
            
        
            
            
    def predict(self,x,y):
        predicted,loss = self.forward_prop(x,y)
        
        predicted = np.around(predicted)
        print("loss =", loss)
        return predicted
            
        
    
    
    
    
    
    
    

In [517]:
model = ann(nn_input,[8,5],nn_output)


(13, 8)
(8, 5)
(5, 1)
[array([ 0.68255141, -0.31011677, -2.43483776,  1.0388246 ,  2.18697965,
        0.44136444, -0.10015523, -0.13644474]), array([-0.11905419,  0.01740941, -1.12201873, -0.51709446, -0.99702683]), array([0.24879916])]


In [518]:
model.train(train_features,train_target)

<ipython-input-516-152224af210b>:33: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


In [519]:
predicted =model.predict(test_features,test_features)

loss = 13601.284863386458


<ipython-input-516-152224af210b>:33: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
